In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!unzip '/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/google-quest-challenge.zip'
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape, df_test.shape

Archive:  /content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/google-quest-challenge.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


((6079, 41), (476, 11))

# 3. Data Preprocessing

In [4]:
# defining a function to remove stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove('no'); stop_words.remove('not'); stop_words.remove('nor')
 
def stopwrd_removal(sent):
  lst = []
  for wrd in sent.split():
    if wrd not in stop_words:
      lst.append(wrd)
  return " ".join(lst)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def text_preprocessor(column, remove_stopwords = False, remove_specialchar = False):
  """pass any column with Text in it from df_train | Note: returns nothing makes inplace changes in df_train"""
  # 1. remove html tags, html urls, replace html comparison operators
  # text = df_train[column].values
  df_train[column] = [re.sub('<.*?>', ' ', i) for i in df_train[column].values]
  df_train[column] = df_train[column].str.replace('&lt;', '<')\
                                          .str.replace('&gt;', '>')\
                                          .str.replace('&le;', '<=' )\
                                          .str.replace('&ge;', '>=')
 
  # 2. remove latex i,e., if there is any formulas or latex we have to remove it
  df_train[column] = [re.sub('\$.*?\$', ' ', i) for i in df_train[column].values]
 
  # 3. all lowercase 
  df_train[column] = df_train[column].str.lower()
 
  # 4. decontractions
  df_train[column] = df_train[column].str.replace("won't", "will not").str.replace("can\'t", "can not").str.replace("n\'t", " not").str.replace("\'re", " are").str.\
                                                replace("\'s", " is").str.replace("\'d", " would").str.replace("\'ll", " will").str.\
                                                replace("\'t", " not").str.replace("\'ve", " have").str.replace("\'m", " am")
  
  # 5. removing non-english or hebrew characters
  df_train[column] = [i.encode("ascii", "ignore").decode() for i in df_train[column].values]
 
  # 6. remove all special-characters other than alpha-numericals
  if remove_specialchar == True:
    df_train[column] = [re.sub('[^A-Za-z0-9]+', ' ', i) for i in df_train[column].values]
 
  # # 7. separating special chars from alphanumerics
  # all_sc = [re.findall('[^ A-Za-z0-9]', i) for i in df_train[column].values]
  # special_char = np.unique([j for i in all_sc for j in i])
  # replace_char = [' '+i+' ' for i in special_char]
  # for i,j in zip(special_char, replace_char):
  #  df_train[column] = df_train[column].str.replace(i, j)
 
  # 8. Stop_word removal
  if remove_stopwords == True:
    df_train[column] = [stopwrd_removal(i) for i in df_train[column].values]
 
  # 9. remove all white-space i.e., \n, \t, and extra_spaces
  df_train[column] = df_train[column].str.replace("\n", " ").str.replace("\t", " ").str.rstrip()
  df_train[column] = [re.sub('  +', ' ', i) for i in df_train[column].values]

In [6]:
# 1. text preprocessing
df_train['clean_title'] = df_train['question_title']
df_train['clean_body'] = df_train['question_body']
df_train['clean_answer'] = df_train['answer']
text_preprocessor('clean_title',  remove_stopwords = False, remove_specialchar = False)
text_preprocessor('clean_body',  remove_stopwords = False, remove_specialchar = False)
text_preprocessor('clean_answer',  remove_stopwords = False, remove_specialchar = False)

# 4. Train_test_split - Random split

In [7]:
# 1. setting up target features
question_tar = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written']
       
answer_tar = ['answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']
 
tar_features = question_tar + answer_tar
len(tar_features)

30

In [8]:
# 2. splitting dataset train_test_split
from sklearn.model_selection import train_test_split
 
X_train, X_cv, y_train, y_cv = train_test_split(df_train[['clean_title', 'clean_body', 'clean_answer']], df_train[tar_features], test_size = 0.12, random_state = 42)
X_train.shape, X_cv.shape, y_train.shape, y_cv.shape

((5349, 3), (730, 3), (5349, 30), (730, 30))

In [9]:
# 3. creating training features : title + body = title_body | answer_train | title + body + answer = title_body_answer
title_train = X_train['clean_title'].values
body_train = X_train['clean_body'].values
answer_train = X_train['clean_answer'].values
 
title_cv = X_cv['clean_title'].values
body_cv = X_cv['clean_body'].values
answer_cv = X_cv['clean_answer'].values
 
# train data
title_body_train = [i+' '+j for i,j in zip(title_train, body_train)]
y_train_ques = y_train[question_tar].values
y_train_ans = y_train[answer_tar].values
 
# cv data
title_body_cv = [i+' '+j for i,j in zip(title_cv, body_cv)]
y_cv_ques = y_cv[question_tar].values
y_cv_ans = y_cv[answer_tar].values
 
len(title_body_train), len(answer_train), len(title_body_cv), len(answer_cv)

(5349, 5349, 730, 730)

# 4.1. Sentence Embedding : BERT

In [10]:
!pip install transformers

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
     |████████████████████████████████| 1.1MB 48.3MB/s 
     |████████████████████████████████| 3.0MB 44.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=57b6c263bc7fe6a54051d4d59400787222b18e9aeb68595aec2c810ac082dc1f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig
 
# Load pretrained model/tokenizer
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Tokenizing : title_body
title_body_train_tokens = bert_tokenizer.batch_encode_plus(title_body_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
title_body_cv_tokens = bert_tokenizer.batch_encode_plus(title_body_cv, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)

tb_train_input_ids  = np.array(title_body_train_tokens['input_ids'])
tb_train_attn_mask = np.array(title_body_train_tokens['attention_mask'])
tb_train_token_typ_ids = np.array(title_body_train_tokens['token_type_ids'])
 
tb_cv_input_ids  = np.array(title_body_cv_tokens['input_ids'])
tb_cv_attn_mask = np.array(title_body_cv_tokens['attention_mask'])
tb_cv_token_typ_ids = np.array(title_body_cv_tokens['token_type_ids'])

tb_train_input_ids.shape, tb_train_attn_mask.shape, tb_train_token_typ_ids.shape, tb_cv_input_ids.shape, tb_cv_attn_mask.shape, tb_cv_token_typ_ids.shape

((5349, 512), (5349, 512), (5349, 512), (730, 512), (730, 512), (730, 512))

In [ ]:
# Tokenizing : answer
ans_train_tokens = bert_tokenizer.batch_encode_plus(answer_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
ans_cv_tokens = bert_tokenizer.batch_encode_plus(answer_cv, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)
 
ans_train_input_ids  = np.array(ans_train_tokens['input_ids'])
ans_train_attn_mask = np.array(ans_train_tokens['attention_mask'])
ans_train_token_typ_ids = np.array(ans_train_tokens['token_type_ids'])
 
ans_cv_input_ids  = np.array(ans_cv_tokens['input_ids'])
ans_cv_attn_mask = np.array(ans_cv_tokens['attention_mask'])
ans_cv_token_typ_ids = np.array(ans_cv_tokens['token_type_ids'])
seq_len = ans_train_input_ids.shape[1]
 
ans_train_input_ids.shape, ans_train_attn_mask.shape, ans_train_token_typ_ids.shape, ans_cv_input_ids.shape, ans_cv_attn_mask.shape, ans_cv_token_typ_ids.shape

((5349, 512), (5349, 512), (5349, 512), (730, 512), (730, 512), (730, 512))

In [14]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
# BERT embedings : title_body_train 
batch_size = 32
l = tb_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = bert_model([tb_train_input_ids[i*batch_size:(i+1)*batch_size], tb_train_attn_mask[i*batch_size:(i+1)*batch_size], tb_train_token_typ_ids[i*batch_size:(i+1)*batch_size]])[2]

  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]

  # concat all 4 averaged hidden_states
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_BERT_train = tf.concat(lst1, axis = 0)
print(tb_BERT_train.shape)

# BERT embedings : title_body_train
batch_size = 32
l = ans_cv_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = bert_model([tb_cv_input_ids[i*batch_size:(i+1)*batch_size], tb_cv_attn_mask[i*batch_size:(i+1)*batch_size], tb_cv_token_typ_ids[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]
  # lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_BERT_cv = tf.concat(lst1, axis = 0)
print(tb_BERT_cv.shape)


(5349, 3072)



(730, 3072)


In [ ]:
# BERT embedings : answer_train
batch_size = 32
l = ans_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = bert_model([ans_train_input_ids[i*batch_size:(i+1)*batch_size], ans_train_attn_mask[i*batch_size:(i+1)*batch_size], ans_train_token_typ_ids[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers
  lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_BERT_train = tf.concat(lst1, axis = 0)
print(ans_BERT_train.shape)

# cv : ans BERT embedings
batch_size = 32
l = ans_cv_input_ids.shape[0]
lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = bert_model([ans_cv_input_ids[i*batch_size:(i+1)*batch_size], ans_cv_attn_mask[i*batch_size:(i+1)*batch_size], ans_cv_token_typ_ids[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers
  lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_BERT_cv = tf.concat(lst1, axis = 0)
print(ans_BERT_cv.shape)


(5349, 3072)



(730, 3072)


In [ ]:
# saving pretrained BERT embeddings
np.savez_compressed('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/bert_outputs', tb_BERT_train = tb_BERT_train.numpy(), tb_BERT_cv = tb_BERT_cv.numpy(), 
                    ans_BERT_train = ans_BERT_train.numpy(), ans_BERT_cv = ans_BERT_cv.numpy())

### predictor Model :

In [12]:
# loading pretrained BERT embeddings
tb_BERT_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/bert_outputs.npz')['tb_BERT_train']
tb_BERT_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/bert_outputs.npz')['tb_BERT_cv']
ans_BERT_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/bert_outputs.npz')['ans_BERT_train']
ans_BERT_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/bert_outputs.npz')['ans_BERT_cv']

In [20]:
tf.keras.backend.clear_session()
seed = 42

# *-----------------title_body-----------------*
tb_input = Input(name = 'tb_bert_out', shape = (768*4,), dtype = 'float32')
tb_dropout_1 = Dropout(rate = 0.2, seed = seed)(tb_input)
 
tb_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_1)
tb_dropout_2 = Dropout(rate = 0.2, seed = seed)(tb_dense_1)
 
tb_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_2)
tb_dropout_3 = Dropout(rate = 0.1, seed = seed)(tb_dense_2)

tb_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_3)
 
# *-----------------answer_model-----------------*
ans_input = Input(name = 'ans_bert_out', shape = (768*4,), dtype = 'float32')
# concat = tf.concat([tb_out, ans_input], axis = -1) 
ans_dropout_1 = Dropout(rate = 0.2, seed = seed)(ans_input)
 
ans_dense_1 = Dense(units = 2048, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_1)
ans_dropout_2 = Dropout(rate = 0.2, seed = seed)(ans_dense_1)
 
ans_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_2)
ans_dropout_3 = Dropout(rate = 0.1, seed = seed)(ans_dense_2)

ans_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_3)

# *--------------------concat--------------------*
out = tf.concat([tb_out, ans_out], axis = -1) 
 
model = Model(inputs = [tb_input, ans_input], outputs = out)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tb_bert_out (InputLayer)        [(None, 3072)]       0                                            
__________________________________________________________________________________________________
ans_bert_out (InputLayer)       [(None, 3072)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 3072)         0           tb_bert_out[0][0]                
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 3072)         0           ans_bert_out[0][0]               
_______________________________________________________________________________________

In [16]:
# post processing : binning
def return_bins(arr):
  val = np.unique(arr)
  bins = []
  for i in range(len(val)):
    if i > 0:
      bins.append((val[i-1] + val[i])/2)
  return bins
  
unique_val_30 = [np.unique(df_train[tar_features].values[:, i]) for i in range(30)]
bins_30 = [return_bins(df_train[tar_features].values[:, i]) for i in range(30)]
 
def binned_out(y_pred):
  col = y_pred.shape[1]
  final_pred = np.zeros(y_pred.shape)
  for i in range(col):
    idx = np.digitize(y_pred[:, i], bins_30[i])
    final_pred[:, i] = unique_val_30[i][idx]
  return final_pred

In [21]:
# Defining callbacks
!rm -r '/content/saved models'
!rm -r '/content/logs'
!mkdir '/content/saved models'
!mkdir '/content/logs/'
 
# tensorboard callback
import datetime
log_dir="logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1, write_graph=True, write_grads=True)
 
# spearman function
from scipy.stats import pearsonr, spearmanr
def compute_spearman(y_true, y_pred, final_pred):
  col = y_true.shape[1]
  lst = []
  for i in range(col):
    # p = round(spearmanr(y_true[:, i], y_pred[:, i])[0], 5)
    p = round(spearmanr(y_true[:, i], final_pred[:, i])[0], 5)
    p = round(p, 5)
    # if np.isnan(p):
    #   p = round(spearmanr(y_true[:, i], y_pred[:, i])[0], 5)
    lst.append(p)
  return np.array(lst), round(np.nanmean(lst), 5)
 
# Custom spearman metric
class print_spearman(tf.keras.callbacks.Callback):
    def __init__(self, train_data, validation_data):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.x, self.y = train_data
        self.val_x, self.val_y = validation_data
    
    def on_train_begin(self, logs={}):
        self.all_feat_spearman = []
        self.spearman_dict = {'train_spearman' :[], 'val_spearman' :[]}
 
    def on_epoch_end(self, epoch, logs={}):
        self.epoch = epoch
        # 1. Test_set evaluation
        print('\nspearman :')
        y_pred = self.model.predict(x = self.x)
        y_pred_val = self.model.predict(x = self.val_x)
 
        final_pred = binned_out(y_pred)
        final_pred_val = binned_out(y_pred_val)
 
        train_spear_lst, train_spearman = compute_spearman(self.y, y_pred, final_pred)
        val_spear_lst, val_spearman = compute_spearman(self.val_y, y_pred_val, final_pred_val)
 
        self.all_feat_spearman.append({'train_spearman' : train_spear_lst, 'val_spearman' : val_spear_lst})
 
        self.spearman_dict['train_spearman'].append(train_spearman)
        self.spearman_dict['val_spearman'].append(val_spearman)
        prev_epoch_lr  = tf.keras.backend.eval(self.model.optimizer.lr)
        print("train_spearman : {} | val_spearman : {} | Learning_Rate : {}".format(train_spearman, val_spearman, round(prev_epoch_lr, 6)))
        # print('train_spear_lst : ', train_spear_lst, '\n' 'val_spear_lst :', val_spear_lst)
 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor=np.sqrt(0.1), patience=4, verbose=1)
 
checkpt = tf.keras.callbacks.ModelCheckpoint('/content/saved models/weights.-{epoch:03d}-{val_loss:.5f}', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True)
 
print_spearman_fn = print_spearman(train_data = ([tb_BERT_train, ans_BERT_train], y_train.values),
                                 validation_data = ([tb_BERT_cv, ans_BERT_cv], y_cv.values))
callbacks = [print_spearman_fn, reduce_lr, checkpt, tensorboard_callback]

In [22]:
# BERT_model : training a model
tf.keras.backend.clear_session()
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
rmse = tf.keras.metrics.RootMeanSquaredError()
 
model.compile(loss = 'binary_crossentropy', optimizer = opt,  metrics = [rmse])
history = model.fit(x = [tb_BERT_train, ans_BERT_train], y =  y_train.values,
                    validation_data = ([tb_BERT_cv, ans_BERT_cv], y_cv.values),
                    batch_size = 64, epochs = 40, callbacks = callbacks)

Epoch 1/40
84/84 [==============================] - ETA: 0s - loss: 0.4389 - root_mean_squared_error: 0.2535
spearman :
train_spearman : 0.28278 | val_spearman : 0.28746 | Learning_Rate : 9.999999747378752e-05

Epoch 00001: saving model to /content/saved models/weights.-001-0.39854
84/84 [==============================] - 2s 24ms/step - loss: 0.4389 - root_mean_squared_error: 0.2535 - val_loss: 0.3985 - val_root_mean_squared_error: 0.2234
Epoch 2/40
76/84 [==========================>...] - ETA: 0s - loss: 0.4059 - root_mean_squared_error: 0.2309
spearman :
train_spearman : 0.32832 | val_spearman : 0.32837 | Learning_Rate : 9.999999747378752e-05

Epoch 00002: saving model to /content/saved models/weights.-002-0.39364
84/84 [==============================] - 2s 23ms/step - loss: 0.4057 - root_mean_squared_error: 0.2307 - val_loss: 0.3936 - val_root_mean_squared_error: 0.2200
Epoch 3/40
76/84 [==========================>...] - ETA: 0s - loss: 0.3988 - root_mean_squared_error: 0.2261
spear

In [23]:
# best_results : (val_spearman, epoch_no)
max(print_spearman_fn.spearman_dict['val_spearman']), np.argmax(print_spearman_fn.spearman_dict['val_spearman'])

(0.39536, 25)

# 4.2. USE Embeddings

In [24]:
import tensorflow as tf

import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Softmax, GRU, LSTM, RNN, Embedding, Dense, RepeatVector, TimeDistributed, Bidirectional, Concatenate
from tensorflow.keras.models import Model

In [ ]:
# loading USE model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.


In [ ]:
# title_body
tb_USE_train = np.zeros((len(title_body_train), 512))
batch_size = 32
for i in tqdm(range(int(len(title_body_train)/batch_size)+1)):
  tb_USE_train[i*batch_size : (i+1)*batch_size] = use_model(title_body_train[i*batch_size : (i+1)*batch_size]).numpy()
print(tb_USE_train.shape)

tb_USE_cv = np.zeros((len(title_body_cv), 512))
batch_size = 64
for i in tqdm(range(int(len(title_body_cv)/batch_size)+1)):
  tb_USE_cv[i*batch_size : (i+1)*batch_size] = use_model(title_body_cv[i*batch_size : (i+1)*batch_size]).numpy()
print(tb_USE_cv.shape)

In [ ]:
# answer
ans_USE_train = np.zeros((len(answer_train), 512))
batch_size = 32
for i in tqdm(range(int(len(answer_train)/batch_size)+1)):
  ans_USE_train[i*batch_size : (i+1)*batch_size] = use_model(answer_train[i*batch_size : (i+1)*batch_size]).numpy()
print(ans_USE_train.shape)

ans_USE_cv = np.zeros((len(answer_cv), 512))
batch_size = 32
for i in tqdm(range(int(len(answer_cv)/batch_size)+1)):
  ans_USE_cv[i*batch_size : (i+1)*batch_size] = use_model(answer_cv[i*batch_size : (i+1)*batch_size]).numpy()
print(ans_USE_cv.shape)


(5349, 512)



(730, 512)


In [ ]:
# saving pretrained USE embeddings
np.savez_compressed('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/use_outputs', tb_USE_train = tb_USE_train, tb_USE_cv = tb_USE_cv, 
                    ans_USE_train = ans_USE_train, ans_USE_cv = ans_USE_cv)

### Modeling :

In [25]:
# laoding pretrained USE embeddings
tb_USE_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/use_outputs.npz')['tb_USE_train']
tb_USE_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/use_outputs.npz')['tb_USE_cv']
ans_USE_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/use_outputs.npz')['ans_USE_train']
ans_USE_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/use_outputs.npz')['ans_USE_cv']

In [36]:
tf.keras.backend.clear_session()
seed = 42

# *-----------------title_body-----------------*
tb_input = Input(name = 'tb_use_out', shape = (512,), dtype = 'float32')
tb_dropout_1 = Dropout(rate = 0.2, seed = seed)(tb_input)
 
tb_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_1)
tb_dropout_2 = Dropout(rate = 0.2, seed = seed)(tb_dense_1)
 
tb_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_2)
tb_dropout_3 = Dropout(rate = 0.2, seed = seed)(tb_dense_2)

tb_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_3)
 
# *-----------------answer_model-----------------*
ans_input = Input(name = 'ans_use_out', shape = (512,), dtype = 'float32')
concat = tf.concat([tb_out, ans_input], axis = -1) 
ans_dropout_1 = Dropout(rate = 0.2, seed = seed)(concat)
 
ans_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_1)
ans_dropout_2 = Dropout(rate = 0.2, seed = seed)(ans_dense_1)
 
ans_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_2)
ans_dropout_3 = Dropout(rate = 0.2, seed = seed)(ans_dense_2)

ans_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_3)

# *--------------------concat--------------------*
out = tf.concat([tb_out, ans_out], axis = -1) 
 
model = Model(inputs = [tb_input, ans_input], outputs = out)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tb_use_out (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 512)          0           tb_use_out[0][0]                 
__________________________________________________________________________________________________
dense (Dense)                   (None, 1536)         787968      dropout[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1536)         0           dense[0][0]                      
_______________________________________________________________________________________

In [37]:
# Defining callbacks
!rm -r '/content/saved models'
!rm -r '/content/logs'
!mkdir '/content/saved models'
!mkdir '/content/logs/'
 
# tensorboard callback
import datetime
log_dir="logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1, write_graph=True, write_grads=True)
 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor=np.sqrt(0.1), patience=7, verbose=1)
 
checkpt = tf.keras.callbacks.ModelCheckpoint('/content/saved models/weights.-{epoch:03d}-{val_loss:.5f}', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True)
 
print_spearman_fn = print_spearman(train_data = ([tb_USE_train, ans_USE_train], y_train.values),
                                 validation_data = ([tb_USE_cv, ans_USE_cv], y_cv.values))
callbacks = [print_spearman_fn, reduce_lr, checkpt, tensorboard_callback]

In [33]:
# LSTM : training a model
tf.keras.backend.clear_session()
opt = tf.keras.optimizers.Adam(learning_rate = 0.00008)
rmse = tf.keras.metrics.RootMeanSquaredError()

model.compile(loss = 'binary_crossentropy', optimizer = opt,  metrics = [rmse])
history = model.fit(x = [tb_USE_train, ans_USE_train], y =  y_train.values,
                    validation_data = ([tb_USE_cv, ans_USE_cv], y_cv.values),
                    batch_size = 64, epochs = 50, callbacks = callbacks)

Epoch 1/50
75/84 [=========================>....] - ETA: 0s - loss: 0.5626 - root_mean_squared_error: 0.3341
spearman :
train_spearman : 0.04865 | val_spearman : 0.05349 | Learning_Rate : 7.999999797903001e-05

Epoch 00001: saving model to /content/saved models/weights.-001-0.45263
84/84 [==============================] - 1s 13ms/step - loss: 0.5521 - root_mean_squared_error: 0.3271 - val_loss: 0.4526 - val_root_mean_squared_error: 0.2563
Epoch 2/50
75/84 [=========================>....] - ETA: 0s - loss: 0.4317 - root_mean_squared_error: 0.2472
spearman :
train_spearman : 0.16816 | val_spearman : 0.16506 | Learning_Rate : 7.999999797903001e-05

Epoch 00002: saving model to /content/saved models/weights.-002-0.41570
84/84 [==============================] - 1s 10ms/step - loss: 0.4309 - root_mean_squared_error: 0.2466 - val_loss: 0.4157 - val_root_mean_squared_error: 0.2363
Epoch 3/50
76/84 [==========================>...] - ETA: 0s - loss: 0.4146 - root_mean_squared_error: 0.2364
spear

In [34]:
# best_results : (val_spearman, epoch_no)
max(print_spearman_fn.spearman_dict['val_spearman']), np.argmax(print_spearman_fn.spearman_dict['val_spearman'])

(0.43661, 48)

# 4.3. Sentence Embedding : RoBERTa

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 14.3MB/s 
     |████████████████████████████████| 3.0MB 20.3MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a6d1c1aca66db3ca7511a0e432fccbe854e961b27b22d704870ee4ed800338d0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
from transformers import RobertaConfig, RobertaTokenizer, TFRobertaModel
 
# Load pretrained model/tokenizer
config = RobertaConfig.from_pretrained('roberta-base', output_hidden_states=True)
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base', config = config)

- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
# Tokenizing : title_body
title_body_train_tokens = roberta_tokenizer.batch_encode_plus(title_body_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
title_body_cv_tokens = roberta_tokenizer.batch_encode_plus(title_body_cv, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)

tb_train_input_ids  = np.array(title_body_train_tokens['input_ids'])
tb_train_attn_mask = np.array(title_body_train_tokens['attention_mask'])
 
tb_cv_input_ids  = np.array(title_body_cv_tokens['input_ids'])
tb_cv_attn_mask = np.array(title_body_cv_tokens['attention_mask'])

print(tb_train_input_ids.shape, tb_train_attn_mask.shape, tb_cv_input_ids.shape, tb_cv_attn_mask.shape)

# Tokenizing : answer
ans_train_tokens = roberta_tokenizer.batch_encode_plus(answer_train, max_length = 300, truncation = True, pad_to_max_length = True, return_tensors="tf")
ans_cv_tokens = roberta_tokenizer.batch_encode_plus(answer_cv, max_length = 300, truncation = True, pad_to_max_length = True, return_tensors="tf")
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)
 
ans_train_input_ids  = np.array(ans_train_tokens['input_ids'])
ans_train_attn_mask = np.array(ans_train_tokens['attention_mask'])
 
ans_cv_input_ids  = np.array(ans_cv_tokens['input_ids'])
ans_cv_attn_mask = np.array(ans_cv_tokens['attention_mask'])
seq_len = ans_train_input_ids.shape[1]
 
print(ans_train_input_ids.shape, ans_train_attn_mask.shape, ans_cv_input_ids.shape, ans_cv_attn_mask.shape)

(5349, 512) (5349, 512) (730, 512) (730, 512)
(5349, 300) (5349, 300) (730, 300) (730, 300)


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
# train : title_body RoBERTa embedings
batch_size = 32
l = tb_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = roberta_model([tb_train_input_ids[i*batch_size:(i+1)*batch_size], tb_train_attn_mask[i*batch_size:(i+1)*batch_size]])[2]

  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [i[:, 0, :] for i in x[-4:]]

  # concat all 4 averaged hidden_states
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_RoBERTa_train = tf.concat(lst1, axis = 0)
print(tb_RoBERTa_train.shape)

# cv : title_body RoBERTa embedings
batch_size = 32
l = ans_cv_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = roberta_model([tb_cv_input_ids[i*batch_size:(i+1)*batch_size], tb_cv_attn_mask[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [i[:, 0, :] for i in x[-4:]]
  # lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_RoBERTa_cv = tf.concat(lst1, axis = 0)
print(tb_RoBERTa_cv.shape)


(5349, 3072)



(730, 3072)


In [ ]:
# train : ans RoBERTa embedings
batch_size = 32
l = ans_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = roberta_model([ans_train_input_ids[i*batch_size:(i+1)*batch_size], ans_train_attn_mask[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_RoBERTa_train = tf.concat(lst1, axis = 0)
print(ans_RoBERTa_train.shape)

# cv : ans BERT embedings
batch_size = 32
l = ans_cv_input_ids.shape[0]
lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = roberta_model([ans_cv_input_ids[i*batch_size:(i+1)*batch_size], ans_cv_attn_mask[i*batch_size:(i+1)*batch_size]])[2]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_RoBERTa_cv = tf.concat(lst1, axis = 0)
print(ans_RoBERTa_cv.shape)


(5349, 3072)



(730, 3072)


In [ ]:
# saving roberta pretrained embeddings
np.savez_compressed('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/roberta_outputs', tb_RoBERTa_train = tb_RoBERTa_train.numpy(), tb_RoBERTa_cv = tb_RoBERTa_cv.numpy(), 
                    ans_RoBERTa_train = ans_RoBERTa_train.numpy(), ans_RoBERTa_cv = ans_RoBERTa_cv.numpy())

### Modelling :

In [35]:
# loading roberta pretrained embeddings
tb_RoBERTa_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/roberta_outputs.npz')['tb_RoBERTa_train']
tb_RoBERTa_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/roberta_outputs.npz')['tb_RoBERTa_cv']
ans_RoBERTa_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/roberta_outputs.npz')['ans_RoBERTa_train']
ans_RoBERTa_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/roberta_outputs.npz')['ans_RoBERTa_cv']

In [44]:
tf.keras.backend.clear_session()
seed = 42

# *-----------------title_body-----------------*
tb_input = Input(name = 'tb_roberta_out', shape = (768*4,), dtype = 'float32')
tb_dropout_1 = Dropout(rate = 0.2, seed = seed)(tb_input)
 
tb_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_1)
tb_dropout_2 = Dropout(rate = 0.2, seed = seed)(tb_dense_1)
 
tb_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_2)
tb_dropout_3 = Dropout(rate = 0.1, seed = seed)(tb_dense_2)

tb_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_3)
 
# *-----------------answer_model-----------------*
ans_input = Input(name = 'ans_roberta_out', shape = (768*4,), dtype = 'float32')
concat = tf.concat([tb_out, ans_input], axis = -1) 
ans_dropout_1 = Dropout(rate = 0.2, seed = seed)(concat)
 
ans_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_1)
ans_dropout_2 = Dropout(rate = 0.2, seed = seed)(ans_dense_1)
 
ans_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_2)
ans_dropout_3 = Dropout(rate = 0.1, seed = seed)(ans_dense_2)

ans_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_3)

# *--------------------concat--------------------*
out = tf.concat([tb_out, ans_out], axis = -1) 
 
model = Model(inputs = [tb_input, ans_input], outputs = out)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tb_roberta_out (InputLayer)     [(None, 3072)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 3072)         0           tb_roberta_out[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         3146752     dropout[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1024)         0           dense[0][0]                      
_______________________________________________________________________________________

In [45]:
# Defining callbacks
!rm -r '/content/saved models'
!rm -r '/content/logs'
!mkdir '/content/saved models'
!mkdir '/content/logs/'
 
# tensorboard callback
import datetime
log_dir="logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1, write_graph=True, write_grads=True)
 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor=np.sqrt(0.1), patience=7, verbose=1)
 
checkpt = tf.keras.callbacks.ModelCheckpoint('/content/saved models/weights.-{epoch:03d}-{val_loss:.5f}', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True)
 
print_spearman_fn = print_spearman(train_data = ([tb_RoBERTa_train, ans_RoBERTa_train], y_train.values),
                                 validation_data = ([tb_RoBERTa_cv, ans_RoBERTa_cv], y_cv.values))
callbacks = [print_spearman_fn, reduce_lr, checkpt, tensorboard_callback]

In [47]:
# LSTM : training a model
tf.keras.backend.clear_session()
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
rmse = tf.keras.metrics.RootMeanSquaredError()
 
model.compile(loss = 'binary_crossentropy', optimizer = opt,  metrics = [rmse])
history = model.fit(x = [tb_RoBERTa_train, ans_RoBERTa_train], y =  y_train.values,
                    validation_data = ([tb_RoBERTa_cv, ans_RoBERTa_cv], y_cv.values),
                    batch_size = 64, epochs = 75, callbacks = callbacks)

Epoch 1/75
77/84 [==========================>...] - ETA: 0s - loss: 0.4304 - root_mean_squared_error: 0.2495
spearman :
train_spearman : 0.24809 | val_spearman : 0.24901 | Learning_Rate : 0.0010000000474974513

Epoch 00001: saving model to /content/saved models/weights.-001-0.41228
84/84 [==============================] - 2s 20ms/step - loss: 0.4291 - root_mean_squared_error: 0.2484 - val_loss: 0.4123 - val_root_mean_squared_error: 0.2344
Epoch 2/75
83/84 [============================>.] - ETA: 0s - loss: 0.4070 - root_mean_squared_error: 0.2320
spearman :
train_spearman : 0.27542 | val_spearman : 0.27254 | Learning_Rate : 0.0010000000474974513

Epoch 00002: saving model to /content/saved models/weights.-002-0.40281
84/84 [==============================] - 2s 18ms/step - loss: 0.4069 - root_mean_squared_error: 0.2320 - val_loss: 0.4028 - val_root_mean_squared_error: 0.2284
Epoch 3/75
82/84 [============================>.] - ETA: 0s - loss: 0.4019 - root_mean_squared_error: 0.2281
spear

In [48]:
# best_results : (val_spearman, epoch_no)
max(print_spearman_fn.spearman_dict['val_spearman']), np.argmax(print_spearman_fn.spearman_dict['val_spearman'])

(0.41772, 58)

# 4.4. Sentence Embedding : XLNet

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 14.3MB/s 
     |████████████████████████████████| 3.0MB 20.3MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a6d1c1aca66db3ca7511a0e432fccbe854e961b27b22d704870ee4ed800338d0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [61]:
import tensorflow as tf
from transformers import XLNetConfig, XLNetTokenizer, TFXLNetModel
 
# Load pretrained model/tokenizer
config = XLNetConfig.from_pretrained('xlnet-base-cased', output_hidden_states=True)
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = TFXLNetModel.from_pretrained('xlnet-base-cased', config = config)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [62]:
# Tokenizing : title_body
title_body_train_tokens = xlnet_tokenizer.batch_encode_plus(title_body_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
title_body_cv_tokens = xlnet_tokenizer.batch_encode_plus(title_body_cv, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)

tb_train_input_ids  = np.array(title_body_train_tokens['input_ids'])
tb_train_attn_mask = np.array(title_body_train_tokens['attention_mask'])
tb_train_token_typ_ids = np.array(title_body_train_tokens['token_type_ids'])
 
tb_cv_input_ids  = np.array(title_body_cv_tokens['input_ids'])
tb_cv_attn_mask = np.array(title_body_cv_tokens['attention_mask'])
tb_cv_token_typ_ids = np.array(title_body_cv_tokens['token_type_ids'])

print(tb_train_input_ids.shape, tb_train_attn_mask.shape, tb_train_token_typ_ids.shape, tb_cv_input_ids.shape, tb_cv_attn_mask.shape, tb_cv_token_typ_ids.shape)

# Tokenizing : answer
ans_train_tokens = xlnet_tokenizer.batch_encode_plus(answer_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
ans_cv_tokens = xlnet_tokenizer.batch_encode_plus(answer_cv, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
ans_train_input_ids  = np.array(ans_train_tokens['input_ids'])
ans_train_attn_mask = np.array(ans_train_tokens['attention_mask'])
ans_train_token_typ_ids = np.array(ans_train_tokens['token_type_ids'])
 
ans_cv_input_ids  = np.array(ans_cv_tokens['input_ids'])
ans_cv_attn_mask = np.array(ans_cv_tokens['attention_mask'])
ans_cv_token_typ_ids = np.array(ans_cv_tokens['token_type_ids'])
seq_len = ans_train_input_ids.shape[1]
 
print(ans_train_input_ids.shape, ans_train_attn_mask.shape, ans_train_token_typ_ids.shape, ans_cv_input_ids.shape, ans_cv_attn_mask.shape, ans_cv_token_typ_ids.shape)

(5349, 512) (5349, 512) (5349, 512) (730, 512) (730, 512) (730, 512)
(5349, 512) (5349, 512) (5349, 512) (730, 512) (730, 512) (730, 512)


In [63]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

In [65]:
# train : title_body XLNet embeddings
batch_size = 32
l = tb_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = xlnet_model([tb_train_input_ids[i*batch_size:(i+1)*batch_size], tb_train_attn_mask[i*batch_size:(i+1)*batch_size]])[1]

  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]

  # concat all 4 averaged hidden_states
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_XLNet_train = tf.concat(lst1, axis = 0)
print(tb_XLNet_train.shape)

# cv : title_body XLNet embeddings
batch_size = 32
l = tb_cv_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = xlnet_model([tb_cv_input_ids[i*batch_size:(i+1)*batch_size], tb_cv_attn_mask[i*batch_size:(i+1)*batch_size]])[1]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]
  # lst = [i[:, 0, :] for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
tb_XLNet_cv = tf.concat(lst1, axis = 0)
print(tb_XLNet_cv.shape)


(5349, 3072)



(730, 3072)


In [66]:
# train : ans XLNet embeddings
batch_size = 32
l = ans_train_input_ids.shape[0]

lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = xlnet_model([ans_train_input_ids[i*batch_size:(i+1)*batch_size], ans_train_attn_mask[i*batch_size:(i+1)*batch_size]])[1]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_XLNet_train = tf.concat(lst1, axis = 0)
print(ans_XLNet_train.shape)

# cv : ans XLNet embeddings
batch_size = 32
l = ans_cv_input_ids.shape[0]
lst1 = []
for i in tqdm(range((l//batch_size)+1)):
  x = xlnet_model([ans_cv_input_ids[i*batch_size:(i+1)*batch_size], ans_cv_attn_mask[i*batch_size:(i+1)*batch_size]])[1]
  # getting hidden states of last 4 layers (avg of each timrstep)
  lst = [tf.reduce_mean(i, axis = 1) for i in x[-4:]]
  conc = tf.concat(lst, axis = 1)
  lst1.append(conc)
ans_XLNet_cv = tf.concat(lst1, axis = 0)
print(ans_XLNet_cv.shape)


(5349, 3072)



(730, 3072)


In [ ]:
# saving pretrained xlnet embeddings
np.savez_compressed('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/xlnet_outputs', tb_XLNet_train = tb_XLNet_train.numpy(), tb_XLNet_cv = tb_XLNet_cv.numpy(), 
                    ans_XLNet_train = ans_XLNet_train.numpy(), ans_XLNet_cv = ans_XLNet_cv.numpy())

### Modelling :

In [81]:
# loading pretrained xlnet embeddings
tb_XLNet_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/xlnet_outputs.npz')['tb_XLNet_train']
tb_XLNet_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/xlnet_outputs.npz')['tb_XLNet_cv']
ans_XLNet_train = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/xlnet_outputs.npz')['ans_XLNet_train']
ans_XLNet_cv = np.load('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/xlnet_outputs.npz')['ans_XLNet_cv']

In [82]:
tf.keras.backend.clear_session()
seed = 42

# *-----------------title_body-----------------*
tb_input = Input(name = 'tb_roberta_out', shape = (768*4,), dtype = 'float32')
tb_dropout_1 = Dropout(rate = 0.2, seed = seed)(tb_input)
 
tb_dense_1 = Dense(units = 1536, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_1)
tb_dropout_2 = Dropout(rate = 0.2, seed = seed)(tb_dense_1)
 
tb_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_2)
tb_dropout_3 = Dropout(rate = 0.1, seed = seed)(tb_dense_2)

tb_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_dropout_3)
 
# *-----------------answer_model-----------------*
ans_input = Input(name = 'ans_roberta_out', shape = (768*4,), dtype = 'float32')
# concat = tf.concat([tb_out, ans_input], axis = -1) 
ans_dropout_1 = Dropout(rate = 0.2, seed = seed)(ans_input)
 
ans_dense_1 = Dense(units = 1024, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_1)
ans_dropout_2 = Dropout(rate = 0.2, seed = seed)(ans_dense_1)
 
ans_dense_2 = Dense(units = 512, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_2)
ans_dropout_3 = Dropout(rate = 0.1, seed = seed)(ans_dense_2)

ans_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_dropout_3)

# *--------------------concat--------------------*
out = tf.concat([tb_out, ans_out], axis = -1) 
 
model = Model(inputs = [tb_input, ans_input], outputs = out)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tb_roberta_out (InputLayer)     [(None, 3072)]       0                                            
__________________________________________________________________________________________________
ans_roberta_out (InputLayer)    [(None, 3072)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 3072)         0           tb_roberta_out[0][0]             
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 3072)         0           ans_roberta_out[0][0]            
_______________________________________________________________________________________

In [83]:
# Defining callbacks
!rm -r '/content/saved models'
!rm -r '/content/logs'
!mkdir '/content/saved models'
!mkdir '/content/logs/'
 
# tensorboard callback
import datetime
log_dir="logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1, write_graph=True, write_grads=True)
 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor=np.sqrt(0.1), patience=7, verbose=1)
 
checkpt = tf.keras.callbacks.ModelCheckpoint('/content/saved models/weights.-{epoch:03d}-{val_loss:.5f}', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True)
 
print_spearman_fn = print_spearman(train_data = ([tb_XLNet_train, ans_XLNet_train], y_train.values),
                                 validation_data = ([tb_XLNet_cv, ans_XLNet_cv], y_cv.values))
callbacks = [print_spearman_fn, reduce_lr, checkpt, tensorboard_callback]

In [84]:
# LSTM : training a model
tf.keras.backend.clear_session()
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
rmse = tf.keras.metrics.RootMeanSquaredError()
 
model.compile(loss = 'binary_crossentropy', optimizer = opt,  metrics = [rmse])
history = model.fit(x = [tb_XLNet_train, ans_XLNet_train], y =  y_train.values,
                    validation_data = ([tb_XLNet_cv, ans_XLNet_cv], y_cv.values),
                    batch_size = 64, epochs = 75, callbacks = callbacks)

Epoch 1/75
77/84 [==========================>...] - ETA: 0s - loss: 0.7246 - root_mean_squared_error: 0.3367
spearman :
train_spearman : 0.16392 | val_spearman : 0.15531 | Learning_Rate : 9.999999747378752e-05

Epoch 00001: saving model to /content/saved models/weights.-001-0.42974
84/84 [==============================] - 2s 23ms/step - loss: 0.7093 - root_mean_squared_error: 0.3333 - val_loss: 0.4297 - val_root_mean_squared_error: 0.2433
Epoch 2/75
84/84 [==============================] - ETA: 0s - loss: 0.4871 - root_mean_squared_error: 0.2764
spearman :
train_spearman : 0.17559 | val_spearman : 0.17226 | Learning_Rate : 9.999999747378752e-05

Epoch 00002: saving model to /content/saved models/weights.-002-0.42308
84/84 [==============================] - 2s 20ms/step - loss: 0.4871 - root_mean_squared_error: 0.2764 - val_loss: 0.4231 - val_root_mean_squared_error: 0.2395
Epoch 3/75
76/84 [==========================>...] - ETA: 0s - loss: 0.4495 - root_mean_squared_error: 0.2585
spear

In [85]:
# best_results : (val_spearman, epoch_no)
max(print_spearman_fn.spearman_dict['val_spearman']), np.argmax(print_spearman_fn.spearman_dict['val_spearman'])

(0.37525, 64)

### # Final Results :


In [90]:
# Final Results
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Model", "train_spearman", "val_spearman"]
x.add_row(["baseline_LSTM", 0.4995, 0.34899])
x.add_row(["BERT", 0.50081, 0.39536])
x.add_row(["USE",  0.52245, 0.4366])
x.add_row(["RoBERTa", 0.48059, 0.41772])
x.add_row(["XLNet", 0.52237, 0.37525])

print(x)

+---------------+----------------+--------------+
|     Model     | train_spearman | val_spearman |
+---------------+----------------+--------------+
| baseline_LSTM |     0.4995     |   0.34899    |
|      BERT     |    0.50081     |   0.39536    |
|      USE      |    0.52245     |    0.4366    |
|    RoBERTa    |    0.48059     |   0.41772    |
|     XLNet     |    0.52237     |   0.37525    |
+---------------+----------------+--------------+
